# Khai báo thư viện và đọc dữ liệu

In [2]:
import pandas as pd
df=pd.read_csv('cleaned_schedules.csv')

# 1. Mô hình dự đoán thời gian đến

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Đọc lại dữ liệu
df = pd.read_csv("cleaned_schedules.csv")

# Chuyển đổi dữ liệu thời gian thành phút
def time_to_minutes(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 60 + m + s / 60

df['Arrival'] = df['Arrival'].apply(time_to_minutes)

# Chuyển đổi cột văn bản thành số
for col in ['Train Name', 'Station Name', 'Station Code', 'Train Number']:
    df[col] = pd.factorize(df[col])[0]

X = df[['Day', 'Train Name', 'Station Name', 'Station Code', 'Id', 'Train Number']]
y_arrival = df['Arrival']

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Tách tập train và test
X_train, X_test, y_train_arrival, y_test_arrival = train_test_split(X, y_arrival, test_size=0.2, random_state=42)

# Xây dựng mô hình
arrival_model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Biên dịch mô hình
arrival_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Huấn luyện với EarlyStopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
arrival_model.fit(X_train, y_train_arrival, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Lưu mô hình
arrival_model.save("arrival_model.keras")


Epoch 1/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 231364.2500 - mae: 377.8044 - val_loss: 123372.8281 - val_mae: 294.1829
Epoch 2/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 122861.4688 - mae: 294.3151 - val_loss: 123129.5547 - val_mae: 295.0039
Epoch 3/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 122228.2266 - mae: 293.2225 - val_loss: 122586.1484 - val_mae: 294.2594
Epoch 4/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 122090.3047 - mae: 293.1004 - val_loss: 122358.2812 - val_mae: 293.8248
Epoch 5/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 121616.6797 - mae: 292.3699 - val_loss: 122183.7812 - val_mae: 292.8463
Epoch 6/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 121910.6797 - mae: 292.5483 - val_loss: 122212.8047 - val_mae: 293.7428
Epoch 7/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 121648.4297 - mae: 292.3108 - val_loss: 122174.6250 - val_mae: 294.8517
Epoch 8/50
3842/3842 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step 

# 2. Mô hình dự đoán thời gian đi

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu
df = pd.read_csv("cleaned_schedules.csv")

# Chuyển đổi thời gian thành phút
def time_to_minutes(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 60 + m + s / 60

df['Departure'] = df['Departure'].apply(time_to_minutes)

# Chuyển đổi cột văn bản thành số
label_encoders = {}
for col in ['Train Name', 'Station Name', 'Station Code', 'Train Number']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Kiểm tra và xử lý giá trị null
df['Departure'] = pd.to_numeric(df['Departure'], errors='coerce')
df.dropna(inplace=True)

X = df[['Day', 'Train Name', 'Station Name', 'Station Code', 'Id', 'Train Number']]
y_departure = df['Departure']

# Tách dữ liệu
X_train, X_test, y_train_departure, y_test_departure = train_test_split(X, y_departure, test_size=0.2, random_state=42)

# Tạo mô hình
departure_model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

departure_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Sử dụng EarlyStopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Huấn luyện mô hình
departure_model.fit(X_train, y_train_departure, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Lưu mô hình
departure_model.save("departure_model.keras")


Epoch 1/50
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - loss: 2207641.2500 - mae: 534.6245 - val_loss: 211709.8125 - val_mae: 386.9337
Epoch 2/50
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - loss: 210627.1406 - mae: 386.9201 - val_loss: 221354.8906 - val_mae: 392.3831
Epoch 3/50
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 205641.9375 - mae: 383.5847 - val_loss: 202594.5938 - val_mae: 382.5066
Epoch 4/50
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - loss: 203495.8750 - mae: 382.3036 - val_loss: 203691.3594 - val_mae: 382.6535
Epoch 5/50
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - loss: 201928.5781 - mae: 380.9024 - val_loss: 202651.9844 - val_mae: 383.2631
Epoch 6/50
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - loss: 201345.4844 - mae: 380.6116 - val_loss: 204705.6406 - val_mae: 383.2298


# 3. Mô hình phát hiện sự chậm trễ

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Đọc lại dữ liệu
df = pd.read_csv("cleaned_schedules.csv")

# Chuyển đổi dữ liệu thời gian thành phút
def time_to_minutes(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 60 + m + s / 60  # Tính toán thời gian thành phút

df['Arrival'] = df['Arrival'].apply(time_to_minutes)
df['Departure'] = df['Departure'].apply(time_to_minutes)

# Chuyển đổi cột văn bản thành số
for col in ['Train Name', 'Station Name', 'Station Code', 'Train Number']:
    df[col] = pd.factorize(df[col])[0]

# Giả định rằng "Delay" được xác định bằng cách so sánh Arrival với Departure
df['Delay'] = (df['Arrival'] > df['Departure']).astype(int)  # 1 nếu có độ trễ, 0 nếu không

# Kiểm tra và chuyển đổi kiểu dữ liệu
df['Delay'] = pd.to_numeric(df['Delay'], errors='coerce')
df.dropna(inplace=True)  # Xóa các dòng có giá trị null

# Chọn dữ liệu đầu vào và đầu ra
X = df[['Day', 'Train Name', 'Station Name', 'Station Code', 'Id', 'Train Number']]
y_delay = df['Delay']

# Tách dữ liệu
X_train, X_test, y_train_delay, y_test_delay = train_test_split(X, y_delay, test_size=0.2, random_state=42)

# Tạo mô hình
delay_model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),  # Thêm lớp BatchNormalization
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile với thêm callback EarlyStopping
delay_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Huấn luyện mô hình
delay_model.fit(X_train, y_train_delay, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
delay_model.save("delay_model.keras")


Epoch 1/10
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.9986 - loss: 0.0095 - val_accuracy: 0.9997 - val_loss: 0.0027
Epoch 2/10
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9996 - loss: 0.0040 - val_accuracy: 0.9997 - val_loss: 0.0026
Epoch 3/10
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - accuracy: 0.9997 - loss: 0.0034 - val_accuracy: 0.9997 - val_loss: 0.0026
Epoch 4/10
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9997 - val_loss: 0.0028
Epoch 5/10
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9996 - loss: 0.0036 - val_accuracy: 0.9997 - val_loss: 0.0027
Epoch 6/10
7683/7683 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9997 - val_loss: 0.0027


# 4. Mô hình phân loại các tuyến đường phổ biến

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu
df = pd.read_csv("cleaned_schedules.csv")

# Chọn các cột đầu vào
X = df[['Day', 'Train Name', 'Station Name', 'Station Code', 'Id', 'Train Number']]

# Xây dựng pipeline tiền xử lý cho các cột chuỗi và số
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), make_column_selector(dtype_include='number')),
    ('cat', OneHotEncoder(), make_column_selector(dtype_include='object'))
])


X = preprocessor.fit_transform(X)

# KMeans với n_init=10 để cải thiện tốc độ hội tụ
kmeans = KMeans(n_clusters=5, n_init=10, random_state=42)
df['Route_Cluster'] = kmeans.fit_predict(X)

# Tách dữ liệu cho việc kiểm tra
y_route_cluster = df['Route_Cluster']
X_train, X_test, y_train_route, y_test_route = train_test_split(X, y_route_cluster, test_size=0.2, random_state=42)

# Xây dựng mô hình với số lớp neuron giảm nhẹ
route_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(5, activation='softmax')
])

# Biên dịch và huấn luyện mô hình
route_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
route_model.fit(X_train, y_train_route, epochs=10, validation_split=0.2, verbose=2)
route_model.save("route_model.keras")


Epoch 1/10
7683/7683 - 162s - 21ms/step - accuracy: 0.9949 - loss: 0.0220 - val_accuracy: 0.9997 - val_loss: 0.0012
Epoch 2/10
7683/7683 - 160s - 21ms/step - accuracy: 1.0000 - loss: 4.6816e-06 - val_accuracy: 1.0000 - val_loss: 3.5915e-05
Epoch 3/10
7683/7683 - 137s - 18ms/step - accuracy: 1.0000 - loss: 2.0560e-07 - val_accuracy: 1.0000 - val_loss: 2.8029e-05
Epoch 4/10
7683/7683 - 117s - 15ms/step - accuracy: 1.0000 - loss: 1.1601e-08 - val_accuracy: 1.0000 - val_loss: 4.2576e-05
Epoch 5/10
7683/7683 - 104s - 14ms/step - accuracy: 1.0000 - loss: 2.5895e-10 - val_accuracy: 1.0000 - val_loss: 3.7059e-05
Epoch 6/10
7683/7683 - 108s - 14ms/step - accuracy: 1.0000 - loss: 7.2740e-11 - val_accuracy: 1.0000 - val_loss: 3.5986e-05
Epoch 7/10
7683/7683 - 112s - 15ms/step - accuracy: 1.0000 - loss: 3.9279e-11 - val_accuracy: 1.0000 - val_loss: 3.4660e-05
Epoch 8/10
7683/7683 - 133s - 17ms/step - accuracy: 1.0000 - loss: 2.5701e-11 - val_accuracy: 1.0000 - val_loss: 3.3341e-05
Epoch 9/10
7683/

# 5. Mô hình dự đoán số lượng hành khách

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Đọc lại dữ liệu
df = pd.read_csv("cleaned_schedules.csv")

# Giả định cột Passenger_Count (nếu không có sẵn)
df['Passenger_Count'] = np.random.randint(50, 200, size=len(df))

# Chọn cột đầu vào và đầu ra
X = df[['Day', 'Train Name', 'Station Name', 'Station Code', 'Id', 'Train Number']]
y_passenger_count = df['Passenger_Count']

# Sử dụng LabelEncoder để mã hóa cột văn bản
label_encoders = {}
for col in ['Train Name', 'Station Name', 'Station Code', 'Train Number']:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Xóa các giá trị NaN nếu có
X = X.fillna(0)
y_passenger_count = y_passenger_count.fillna(0)

# Chuyển đổi kiểu dữ liệu sang int
X = X.astype(int)
y_passenger_count = y_passenger_count.astype(int)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train_passenger, y_test_passenger = train_test_split(X, y_passenger_count, test_size=0.2, random_state=42)

# Tạo Dataset để huấn luyện nhanh hơn
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train_passenger)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test_passenger)).batch(32)

# Tạo mô hình
passenger_model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1)
])

# Biên dịch mô hình
passenger_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Thiết lập EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Huấn luyện mô hình với EarlyStopping
passenger_model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[early_stop])

# Lưu mô hình
passenger_model.save("passenger_model.keras")


Epoch 1/10
9603/9603 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - loss: 5141.8965 - mae: 56.9588 - val_loss: 3096.9153 - val_mae: 37.8433
Epoch 2/10
9603/9603 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - loss: 1878.8481 - mae: 37.5329 - val_loss: 1996.6606 - val_mae: 37.6754
Epoch 3/10
9603/9603 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - loss: 1878.5814 - mae: 37.5320 - val_loss: 1898.3604 - val_mae: 37.6219
Epoch 4/10
9603/9603 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 1878.5028 - mae: 37.5312 - val_loss: 1944.2382 - val_mae: 37.6566
Epoch 5/10
9603/9603 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 1878.4363 - mae: 37.5307 - val_loss: 2438.4094 - val_mae: 37.7736
Epoch 6/10
9603/9603 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - loss: 1878.4065 - mae: 37.5308 - val_loss: 1898.8052 - val_mae: 37.6150
